In [57]:
import altair as alt
import polars as pl
import numpy as np
from sktime.forecasting.compose import make_reduction

In [18]:
X = pl.read_csv("X_train.csv").with_columns(
    pl.col("Time").str.to_datetime("%d/%m/%Y %H:%M")
)
y = pl.read_csv("Y_train.csv").join(X, on="ID").select("ID", "Time", "Production")

X_WF1 = X.filter(pl.col("WF") == "WF1")
y_WF1 = y.join(X_WF1, on="ID", how="semi")

from sktime.split import temporal_train_test_split

y_train, y_test, X_train, X_test = temporal_train_test_split(
    y=y_WF1["Production"].to_pandas(), X=X_WF1.drop(["ID", "WF", "Time"]).to_pandas()
)

fh = np.arange(1, len(y_test) + 1)  # forecasting horizon

# y_WF1_train = y_train.join(X_WF1_train, on="ID", how="semi")
# y_WF1_test = y_test.join(X_WF1_test, on="ID", how="semi")
# y_WF1_true = np.array(y_WF1_test["Production"])

0        0.02
1        0.07
2        0.22
3        0.39
4        0.41
         ... 
37370    0.04
37371    0.33
37372    0.13
37373    0.01
37374    0.00
Name: Production, Length: 37375, dtype: float64

In [20]:
def cape(y_true, y_pred):
    """Compute the cumulated absolute percentage error."""
    return 100 * sum(abs(y_true - y_pred)) / sum(y_true)

In [50]:
capes = {}

## Baseline: Average production

In [24]:
y_pred = np.full_like(y_test, fill_value=y_train.mean())

In [52]:
capes["baseline"] = cape(y_test, y_pred)
cape(y_test, y_pred)

88.32878349994141

## HistGradientBoostingRegressor

In [36]:
from sklearn.ensemble import HistGradientBoostingRegressor

regressor = HistGradientBoostingRegressor()
forecaster_hgbr = make_reduction(
    regressor,
    strategy="recursive",
    window_length=24,
)

In [37]:
forecaster_hgbr.fit(fh=fh, y=y_train)

RecursiveTabularRegressionForecaster(estimator=HistGradientBoostingRegressor(),
                                     window_length=24)

In [39]:
y_pred_hgbr = forecaster_hgbr.predict(fh=fh)

In [53]:
capes["hgbr"] = cape(y_test, y_pred_hgbr)
cape(y_test, y_pred_hgbr)

90.8990943442281

## Random Forest

In [43]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor()
forecaster_rf = make_reduction(
    regressor,
    strategy="recursive",
    window_length=12,
)

forecaster_rf.fit(y_train)

RecursiveTabularRegressionForecaster(estimator=RandomForestRegressor(),
                                     window_length=12)

In [44]:
y_pred_rf = forecaster_rf.predict(fh=fh)

In [54]:
capes["rf"] = cape(y_test, y_pred_rf)
cape(y_test, y_pred_rf)

87.1193593530579

## Use the exogenous variables

In [46]:
regressor = RandomForestRegressor()
forecaster_exo = make_reduction(
    regressor,
    strategy="recursive",
    window_length=12,
)

forecaster_exo.fit(y_train)

RecursiveTabularRegressionForecaster(estimator=RandomForestRegressor(),
                                     window_length=12)

In [47]:
forecaster_exo.fit(y=y_train, X=X_train, fh=fh)

RecursiveTabularRegressionForecaster(estimator=RandomForestRegressor(),
                                     window_length=12)

In [48]:
y_pred_exo = forecaster_exo.predict(fh=fh, X=X_test)

In [55]:
capes["rf_exo"] = cape(y_test, y_pred_exo)
cape(y_test, y_pred_exo)

89.46870175778429

In [69]:
alt.Chart(
    pl.DataFrame(capes)
    .transpose(include_header=True)
    .with_columns(
        pl.col("column").alias("experiment"), pl.col("column_0").alias("cape")
    )
).mark_bar().encode(x="experiment", y=alt.Y("cape").title("CAPE (%)")).properties(
    width=500
)

alt.Chart(...)